In [114]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#estaticos_market = pd.read_csv('~/codenation/projeto_final/files/estaticos_market.csv')
train = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
train2 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
estaticos_portfolio1 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
estaticos_portfolio2 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio2.csv')
estaticos_portfolio3 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio3.csv')

In [4]:
#print(estaticos_market.shape)
print(estaticos_portfolio1.shape)
print(estaticos_portfolio2.shape)
print(estaticos_portfolio3.shape)

(555, 182)
(566, 2)
(265, 2)


In [ ]:
percent_missing = train2.isnull().sum() * 100 / len(train2)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing,
                                 'dtypes': train2.dtypes,})
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df.head(181)

In [17]:
indices = pd.Series(train.index, index=train['id']).drop_duplicates()
train = train.set_index('id')
float_cols = train.select_dtypes(include=['float64']).columns
str_cols = train.select_dtypes(include=['object']).columns
train.loc[:, float_cols] = train.loc[:, float_cols].fillna(0)
train.loc[:, str_cols] = train.loc[:, str_cols].fillna('')
train = pd.get_dummies(train, drop_first=True)

In [18]:
cosine_sim = linear_kernel(train, train)
train.reset_index(level=0, inplace=True)

In [5]:
def get_recommendations(id, cosine_sim=cosine_sim):
    idx = indices[id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    id_indices = [i[0] for i in sim_scores]
    return estaticos_portfolio1['id'].iloc[id_indices].values.tolist() 

In [9]:
def get_id(idlist):
    data_day_list = []
    for element in idlist:
        data_day =(estaticos_portfolio1[estaticos_portfolio1.id==element])
        data_day_list.append(data_day)
    final_data_day = pd.concat(data_day_list)
    return final_data_day


In [ ]:
get_id(get_recommendations('7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720'))

In [129]:

train2 = pd.read_csv('~/codenation/projeto_final/files/estaticos_portfolio1.csv')
str_cols = train2.select_dtypes(include=['object']).columns
train2.loc[:, str_cols] = train2.loc[:, str_cols].fillna(train2['id'])
booleandf = train2.select_dtypes(include=[bool]).columns
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
for column in booleandf:
    train2[column] = train2[column].map(booleanDictionary)
train2=  train2.astype(str)
objectdf  = train2.select_dtypes(include=[object]).columns
train2 = train2.replace(to_replace ='nan', value = train2['id'])
for column in objectdf:
    train2[column] = train2[column].str.replace(' ', '')
    train2[column] = np.where(train2[column] == 'nan', train2['id'], train2[column])
    #train2[column] = train2[column].str.replace('nan',train2['id'])
    train2[column] = column+train2[column]
train2 = train2.drop(['Unnamed: 0'],axis=1) 
train2 = train2.drop(['fl_matriz'],axis=1) 

train2['new'] = train2.apply(''.join, axis=1)
train3 = train2.drop(['id'],axis=1) 
cv = CountVectorizer()
count_matrix = cv.fit_transform(train3['new'])
cosine_sim = cosine_similarity(count_matrix)
indices2 = pd.Series(train2.index, index=train2['id']).drop_duplicates()

In [130]:
def get_recommendations2(id, cosine_sim=cosine_sim):
    idx = indices2[id]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    id_indices =[]
    for i in sim_scores:
        if i[1]>=0.5:
            id_indices.append(i[0]) 
    #id_indices = [i[0] for i in sim_scores]
    return estaticos_portfolio1['id'].iloc[id_indices].values.tolist()
def get_id2(idlist):
    data_day_list = []
    for element in idlist:
        data_day =(estaticos_portfolio1[estaticos_portfolio1.id==element])
        data_day_list.append(data_day)
    final_data_day = pd.concat(data_day_list)
    return final_data_day
requis=(['id7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720','id970485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389','id6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570'])
result_list=[]
for element in requis:
    print(element)
    result = get_id2(get_recommendations2(element))
    result_list.append(result)
result_final = pd.concat(result_list)
result_final.drop_duplicates(keep = 'first', inplace = True)
result_final

id7d58990ba786a3a26617a8afbf814a9919ab34aa09013a559b2c9d45eb439720
id970485e6b8abe52d8fb4e0bf060fb6f0f4f8576cb7583d885a8dcac38f506389
id6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570


,Unnamed: 0,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,...,media_meses_servicos,max_meses_servicos,min_meses_servicos,qt_funcionarios,qt_funcionarios_12meses,qt_funcionarios_24meses,tx_crescimento_12meses,tx_crescimento_24meses,tx_rotatividade,qt_filiais
9,9,7d58990ba786a3a26617a8afbf814a9919ab34aa09013a...,True,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,5.309589,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
37,37,d4cb950ffc7f6de0a38403a590bff553a03185b1ec9dac...,True,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,16.350685,15 a 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
426,426,6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c78...,True,SOCIEDADE EMPRESARIA LIMITADA,AM,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,SERVIÇO,6.361644,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
155,155,23be3d5e72271bf16f51c448c5e9b1287e2fa19818221b...,True,SOCIEDADE EMPRESARIA LIMITADA,RO,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,16.523288,15 a 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
133,133,2be489bf8b056c634f8bfd2367d09608b4421e5911fc12...,True,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,3.794521,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
536,536,5bf996b49298eb2fc838bc393d7a17a60d80647340028c...,True,SOCIEDADE EMPRESARIA LIMITADA,AC,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,6.917808,5 a 10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
488,488,dc1827198539361b91487caaa8dc1ae44e9a898aec36c9...,False,SOCIEDADE EMPRESARIA LIMITADA,MA,ENTIDADES EMPRESARIAIS,COMERCIO VAREJISTA,COMERCIO,26.145205,> 20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
528,528,f66944b5f183ab1db5d2688fd960431a1cc467e62ac99a...,True,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AM,OUTROS,COMERCIO VAREJISTA,COMERCIO,0.953425,<= 1,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0
515,515,a910c2add4cb3ce618ca55334e2fe8187d7a85de394c8f...,True,EMPRESA INDIVIDUAL DE RESPONSABILIDADE LIMITAD...,AC,OUTROS,COMERCIO VAREJISTA,COMERCIO,1.895890,1 a 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
353,353,d23652754eb2237240d1a4e4265e891cd109f25bd85d22...,True,SOCIEDADE EMPRESARIA LIMITADA,PI,ENTIDADES EMPRESARIAIS,BENS DE CONSUMO,INDUSTRIA,10.073973,10 a 15,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0


In [122]:
id='id6fecd2c150c5fe474b31b2b4b68e5d7dc9d22f51dd5c781d801b58102a71a570'
idx = indices2[id]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#id_indices = [i[0] for i in sim_scores]
id_indices =[]
for i in sim_scores:
    if i[1]>=0.5:
        print(i[1])
        id_indices.append(i[0]) 
id_indices

0.9999999999999988
0.7313519335211501
0.720596758028192
0.7131292325853255
0.7098415825352339
0.6918679922364658
0.6739528575593802
0.6696652510408083
0.6566920368364693
0.6560657050704434
0.6419407387663691
0.6414560722278401
0.6279794734706597
0.6263878484006787
0.6229601207078626
0.6216038822203298
0.6184880536659318
0.58241017737658
0.581320966363734
0.579014647808332
0.5701831753136356
0.5648790304908825
0.5622618669690291
0.5599527735703037
0.5554186683931089
0.5504690116649318
0.5497252060782751
0.5482198313375874
0.5441736190811486
0.5402581353894291
0.5371517298116736
0.5371517298116736
0.5371517298116736
0.5314805986430001
0.5313148617302947
0.5303948353054367
0.5275443983747535
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.5271681934848628
0.527168193484862

[426,
 9,
 37,
 155,
 536,
 133,
 488,
 528,
 353,
 515,
 141,
 523,
 330,
 78,
 143,
 433,
 413,
 522,
 526,
 8,
 157,
 57,
 391,
 73,
 449,
 470,
 373,
 210,
 441,
 282,
 187,
 310,
 352,
 486,
 462,
 525,
 173,
 6,
 17,
 38,
 97,
 140,
 180,
 195,
 218,
 249,
 281,
 369,
 380,
 381,
 424,
 445,
 504,
 510,
 534,
 540,
 404,
 55,
 41,
 76,
 327,
 410,
 490,
 13,
 25,
 27,
 33,
 50,
 53,
 60,
 71,
 94,
 95,
 109,
 110,
 126,
 136,
 147,
 153,
 158,
 162,
 189,
 204,
 206,
 228,
 241,
 243,
 244,
 248,
 266,
 270,
 279,
 280,
 283,
 293,
 313,
 318,
 319,
 321,
 324,
 336,
 339,
 340,
 344,
 351,
 371,
 376,
 383,
 425,
 444,
 466,
 480,
 481,
 499,
 520,
 524,
 533,
 543,
 548,
 550,
 552,
 14,
 42,
 491,
 91,
 119,
 164,
 203,
 208,
 238,
 329,
 354,
 471,
 80,
 219,
 45,
 62,
 63,
 92,
 188,
 196,
 326,
 356,
 414,
 451,
 487,
 505,
 512,
 529,
 51,
 223,
 308,
 378,
 385,
 417,
 493,
 361,
 96,
 262,
 396,
 476,
 215,
 59,
 84,
 105,
 121,
 255,
 317,
 357,
 368,
 377,
 387,
 418,
